If not using colab, you can comment below out. 
If using Colab, upload rating_1m.csv, CategoricalRBM.py and dataProcess.py.

In [2]:
from google.colab import files
files.upload()

Saving CategoricalRBM.py to CategoricalRBM.py
Saving dataProcess.py to dataProcess.py


{'CategoricalRBM.py': b'import torch\nfrom torch import nn\n\nclass CategoricalRBM(nn.Module):\n\n    def __init__(self, D, M, K):\n        \'\'\'\n        Categorical RBM has a visible part of D features, each feature is a K size one-hot tensor.\n        Hidden part has size M, each is a Bernoulli variable.\n        \'\'\'\n        super(CategoricalRBM, self).__init__()\n        self.W = nn.Parameter(torch.randn(D, K, M))\n        self.c = nn.Parameter(torch.randn(M))\n        self.b = nn.Parameter(torch.randn(D, K))\n        self.hact = nn.Sigmoid()\n        self.vact = nn.Softmax(dim=-1)\n        self.splus = nn.Softplus()\n        self.refloss = nn.CrossEntropyLoss()\n\n\n    def free_energy(self, V):\n        f1 = torch.mean(torch.tensordot(V, self.b, dims=([1,2],[0,1])))\n        f2 = torch.mean(torch.sum(self.splus(torch.tensordot(V, self.W, dims=([1,2],[0,1])) + self.c), dim = 1))\n        return f1 + f2\n\n    def forward(self, Visible, Mask):\n        \'\'\'\n        Visible 

In [3]:
# !rm dataProcess*
# !rm Categorical*
!ls

CategoricalRBM.py  dataProcess.py  __pycache__	rating_1m.csv  sample_data


In [4]:
import torch
from torch import nn
import numpy as np
import CategoricalRBM
import dataProcess
from torchvision import datasets
from torch.utils.data import Dataset, DataLoader

#### DataProcessing

In [5]:
Visible, Mask = dataProcess.getVandMask()

   Unnamed: 0  user  movie  rating
0           0     1   1193       5
1           1     1    661       3
2           2     1    914       3
3           3     1   3408       4
4           4     1   2355       5
1000209


In [6]:
N = len(Visible)
N

6040

In [7]:
cut = int(N*0.8)
Visible_train = Visible[:cut]
Visible_test = Visible[cut:]
Mask_train = Mask[:cut]
Mask_test = Mask[cut:]

In [8]:
data_train = dataProcess.ratingdataset(Visible_train, Mask_train)
data_test = dataProcess.ratingdataset(Visible_test, Mask_test)

In [9]:
bs = 20
train_dataloader = DataLoader(data_train, batch_size=bs, shuffle=True)
test_dataloader = DataLoader(data_test, batch_size=bs, shuffle=True)

#### Model and Training

In [14]:
D, K = Visible.shape[1], Visible.shape[2]
D,K

(3952, 500, 5)

How does the size of the hidden part influence the training?
Below we test with M = 100, 200, 500 respectively.

In [19]:
M = 100
model = CategoricalRBM.CategoricalRBM(D, M, K)
epochs = 10
lr = 0.05
for epoch in range(epochs):
  CategoricalRBM.train_loop(model, train_dataloader, epoch, lr)

Epoch 1 		 Training Loss: (88186.41279602051, 364.4066644463657) 		 Cross Entropy Loss: (tensor(145546.7656), tensor(601.4329))
Epoch 2 		 Training Loss: (41372.30519866943, 170.959938837477) 		 Cross Entropy Loss: (tensor(121434.5391), tensor(501.7956))
Epoch 3 		 Training Loss: (35703.872146606445, 147.53666176283656) 		 Cross Entropy Loss: (tensor(119820.0234), tensor(495.1241))
Epoch 4 		 Training Loss: (32668.76300048828, 134.99488843176977) 		 Cross Entropy Loss: (tensor(120251.8359), tensor(496.9084))
Epoch 5 		 Training Loss: (30291.243103027344, 125.17042604556754) 		 Cross Entropy Loss: (tensor(123454.2031), tensor(510.1413))
Epoch 6 		 Training Loss: (29977.187286376953, 123.87267473709485) 		 Cross Entropy Loss: (tensor(123582.5000), tensor(510.6715))
Epoch 7 		 Training Loss: (29347.197509765625, 121.26941119737862) 		 Cross Entropy Loss: (tensor(125468.4375), tensor(518.4646))
Epoch 8 		 Training Loss: (28780.84292602539, 118.92910300010492) 		 Cross Entropy Loss: (tensor

In [20]:
M = 200
model = CategoricalRBM.CategoricalRBM(D, M, K)
epochs = 10
lr = 0.05
for epoch in range(epochs):
  CategoricalRBM.train_loop(model, train_dataloader, epoch, lr)

Epoch 1 		 Training Loss: (90212.71142578125, 372.7797992800878) 		 Cross Entropy Loss: (tensor(140263.9531), tensor(579.6031))
Epoch 2 		 Training Loss: (38726.63863372803, 160.02743237077698) 		 Cross Entropy Loss: (tensor(115327.1875), tensor(476.5586))
Epoch 3 		 Training Loss: (33485.841957092285, 138.3712477565797) 		 Cross Entropy Loss: (tensor(115425.6484), tensor(476.9655))
Epoch 4 		 Training Loss: (30603.33042907715, 126.46004309536012) 		 Cross Entropy Loss: (tensor(117343.0859), tensor(484.8888))
Epoch 5 		 Training Loss: (29493.11309814453, 121.87236817415095) 		 Cross Entropy Loss: (tensor(120358.7578), tensor(497.3503))
Epoch 6 		 Training Loss: (28155.24870300293, 116.34400290497078) 		 Cross Entropy Loss: (tensor(122287.9688), tensor(505.3222))
Epoch 7 		 Training Loss: (27213.030731201172, 112.45054021157509) 		 Cross Entropy Loss: (tensor(125081.2031), tensor(516.8645))
Epoch 8 		 Training Loss: (26274.310165405273, 108.5715296091127) 		 Cross Entropy Loss: (tensor(

In [16]:
M = 500
model = CategoricalRBM.CategoricalRBM(D, M, K)
epochs = 10
lr = 0.05
for epoch in range(epochs):
  CategoricalRBM.train_loop(model, train_dataloader, epoch, lr)

Epoch 1 		 Training Loss: (105024.35803985596, 433.9849505779172) 		 Cross Entropy Loss: (tensor(149620.1250), tensor(618.2650))
Epoch 2 		 Training Loss: (34202.60576629639, 141.3330816789107) 		 Cross Entropy Loss: (tensor(102855.8516), tensor(425.0242))
Epoch 3 		 Training Loss: (29886.250946044922, 123.49690473572282) 		 Cross Entropy Loss: (tensor(109539.2109), tensor(452.6414))
Epoch 4 		 Training Loss: (27023.647720336914, 111.66796578651618) 		 Cross Entropy Loss: (tensor(115882.8047), tensor(478.8546))
Epoch 5 		 Training Loss: (24886.80712890625, 102.8380459872159) 		 Cross Entropy Loss: (tensor(120505.4453), tensor(497.9564))
Epoch 6 		 Training Loss: (22996.33984375, 95.02619770144628) 		 Cross Entropy Loss: (tensor(124141.7344), tensor(512.9824))
Epoch 7 		 Training Loss: (21517.564056396484, 88.91555395205158) 		 Cross Entropy Loss: (tensor(127116.0156), tensor(525.2728))
Epoch 8 		 Training Loss: (19954.069427490234, 82.45483234500097) 		 Cross Entropy Loss: (tensor(1304